# Burden Analysis

* **Project:** ADRD Genetic Diversity in Biobanks
* **Version:** Python/3.9 and 3.10
* **Last Updated:** 20-FEB-2025

## Notebook Overview
Burden analysis

# Query ADSP to do Burden analysis

In [ ]:
import numpy as np
import pandas as pd
import sys
from functools import reduce
import argparse

In [ ]:
%%bash
module load plink/2.0
plink2 --bfile ${WORK_DIR}/filtered_UNQC_Files --max-maf 0.01 --make-bed --out UnQC_EUR_maf

In [ ]:
%%writefile var_rang.txt
chr1 155234452 155244627
chr1 226870616 226903668 
chr4 89724099 89838304
chr6 41158508 41163116 
chr14 73136417 73223691
chr17 44345302 44353106
chr17 45894554 46028334
chr19 44905796 44909393
chr21 25880550 26171128
chr12 64452120 64502114
chr1 11012654 11025492

In [ ]:
%%bash
module load plink/2.0
plink2 --bfile UnQC_EUR_maf --extract range var_rang.txt --make-bed --out UnQC_EUR_maf_range 

In [ ]:
%%bash
module load plink/2.0
plink2 --bfile UnQC_EUR_maf_range --recode vcf-iid --out UnQC_EUR_maf_range_recode

In [ ]:
! module load annovar 

## Annotation

In [ ]:
%%bash

table_annovar.pl UnQC_EUR_maf_range_recode.vcf $ANNOVAR_DATA/hg38 -buildver hg38 \
--thread 8 \
--out UnQC_EUR_maf_range_recode.annovar \
-remove -protocol refGene,avsnp151,clinvar_20240917,dbnsfp47a,dbnsfp47a_interpro,gnomad41_genome \
-operation g,f,f,f,f,f  \
--nopolish \
-nastring . \
-vcfinput

In [17]:
! grep -v contig UnQC_EUR_maf_range_recode.annovar.hg38_multianno.vcf | grep chr| grep -v intergenic | grep -v intronic | grep -v UTR | grep -v downstream| grep -v ncRNA_exonic | grep -v upstream > UnQC_All_variants

In [ ]:
import pandas as pd
import re
from collections import Counter

filename = "UnQC_All_variants" 

with open(filename, "r") as f:
    lines = f.readlines()

exonic_func_list = []

for line in lines:
    match = re.search(r'ExonicFunc\.refGene=([^;\n]+)', line)  
    if match:
        exonic_func_list.append(match.group(1))  


exonic_func_counts = Counter(exonic_func_list)


summary_df = pd.DataFrame(exonic_func_counts.items(), columns=["Exonic Function", "Count"])


print(summary_df)


summary_df.to_csv("ExonicFunc_Summary.csv", index=False)
print("Summary saved as 'ExonicFunc_Summary.csv'")


In [ ]:
import pandas as pd
import re


filename = "UnQC_All_variants"  


synonymous_variants = []
other_variants = []


with open(filename, "r") as f:
    for line in f:
        match = re.search(r'ExonicFunc\.refGene=([^;\n]+)', line)  
        if match:
            exonic_func = match.group(1)  
            if exonic_func == "synonymous_SNV":
                synonymous_variants.append(line)  
            else:
                other_variants.append(line)  


synonymous_filename = "synonymous_SNV_variants.txt"
other_variants_filename = "Allminus_synonymous_variants.txt"

with open(synonymous_filename, "w") as f:
    f.writelines(synonymous_variants)

with open(other_variants_filename, "w") as f:
    f.writelines(other_variants)

print(f"Synonymous SNV variants saved to: {synonymous_filename}")
print(f"Other variants saved to: {other_variants_filename}")


In [ ]:

input_files = ["synonymous_SNV_variants.txt", "Allminus_synonymous_variants.txt"]
output_files = ["synonymous_SNV_column3.txt", "Allminus_synonymous_variants_column3.txt"]

for i in range(2):
    input_filename = input_files[i]
    output_filename = output_files[i]

    
    with open(input_filename, "r") as infile, open(output_filename, "w") as outfile:
        for line in infile:
            columns = line.strip().split("\t")  
            if len(columns) >= 3:
                outfile.write(columns[2] + "\n")  

    print(f"Extracted third column saved to: {output_filename}")


In [ ]:
%%writefile var_remove2.txt
chr21:25891796:C:T
chr14:73170945:C:T
chr17:46024061:C:T
chr17:44350757:G:A
chr1:11022268:G:A
chr14:73198067:G:A
chr1:11022268:G:A
chr1:155237446:G:T
chr1:155238206:A:C
chr1:155238228:A:G
chr1:155238302:G:A
chr14:73170945:C:T
chr14:73192832:C:A
chr14:73198061:C:T
chr14:73198067:G:A
chr17:44350449:G:GCTGTGAAGACAGGGTGCACTGCTGT
chr17:44350801:G:A
chr17:44351438:G:A
chr17:44352087:C:T
chr17:44352404:C:T
chr17:46024061:C:T
chr21:25891784:C:T
chr21:25891856:C:G
chr12:64497215:G:C
chr12:64455934:A:C
chr12:64474372:G:A
chr12:64488572:GA:G

In [ ]:
%%bash
module load plink/2.0
plink2 --vcf UnQC_EUR_maf_range_recode.annovar.hg38_multianno.vcf  --extract synonymous_SNV_column3.txt --recode vcf --out  UnQC_EUR_maf_range_recode.annovar.hg38_multianno_syn.vcf

In [ ]:
%%bash
module load plink/2.0
plink2 --vcf UnQC_EUR_maf_range_recode.annovar.hg38_multianno.vcf  --extract Allminus_synonymous_variants_column3.txt --exclude var_remove2.txt --recode vcf --out UnQC_EUR_maf_range_recode.annovar.hg38_multianno_allminussyn.vcf

## Burden analysis

In [ ]:
df_covar = pd.read_csv("${WORK_DIR}/covars_alldata_with999forAGRandRACE_PCA.txt", sep="\t")
df_covar

In [ ]:
df_pheno = pd.read_csv(f"${WORK_DIR}/FID_IID_PHENO_EUR.fam", sep=" ")
df_covar_ancestry = df_covar.merge(df_pheno, on=["FID","IID"], how="inner")
df_covar_ancestry.to_csv(f"${WORK_DIR}/burden_covar_EUR.txt", sep="\t", index=False)


In [ ]:
%%bash
module load rvtests/2.1.0

In [ ]:
%%bash
rvtest 

## Synonymous variants

In [37]:
! bgzip  UnQC_EUR_maf_range_recode.annovar.hg38_multianno_syn.vcf.vcf -k

In [38]:
! tabix -f -p vcf UnQC_EUR_maf_range_recode.annovar.hg38_multianno_syn.vcf.vcf.gz

In [ ]:
%%bash
rvtest \
--inVcf UnQC_EUR_maf_range_recode.annovar.hg38_multianno_syn.vcf.vcf.gz \
--out UnQC_Allgenes_syn.burden \
--numThread 10 \
--noweb \
--hide-covar \
--kernel skat,skato \
--pheno "${WORK_DIR}/burden_covar_EUR.txt" \
--pheno-name PHENO \
--geneFile refFlat.txt \
--covar "${WORK_DIR}/burden_covar_EUR.txt" \
--covar-name SEX,AGE,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10 \
--multipleAllele \
--gene APP,PSEN1,PSEN2,TREM2,SNCA,APOE,GRN,MAPT,TARDBP,TBK1,GBA 

## All variants excluding synonymous

In [41]:
! bgzip  UnQC_EUR_maf_range_recode.annovar.hg38_multianno_allminussyn.vcf.vcf -k

In [43]:
! tabix -f -p vcf UnQC_EUR_maf_range_recode.annovar.hg38_multianno_allminussyn.vcf.vcf.gz

In [ ]:
%%bash
rvtest \
--inVcf  UnQc_EUR_maf_range_recode.annovar.hg38_multianno_allminussyn.vcf.vcf.gz \
--out Allgenesminus_syn.burden \
--numThread 10 \
--noweb \
--hide-covar \
--kernel skat,skato \
--pheno "${WORK_DIR}/burden_covar_EUR.txt" \
--pheno-name PHENO \
--geneFile refFlat.txt \
--covar "${WORK_DIR}/burden_covar_EUR.txt" \
--covar-name SEX,AGE,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10 \
--multipleAllele \
--gene APP,PSEN1,PSEN2,TREM2,SNCA,APOE,GRN,MAPT,TARDBP,TBK1,GBA 